In [1]:
import numpy as np
from scipy import optimize
import itertools
import math

In [2]:
def k_LP_original(k,a,eps):
    var_n = int(math.pow(2,k))-1; ub_n = int(math.pow(2,k-1)*k)
    l = np.zeros(k)
    for i in range(k):
        l[i] = int(i)
    p = np.zeros(k)
    p[0] = 1; comb = [0]*(k)
    for i in range(1,k):
        p[i] = p[i-1] + math.comb(k,i)
        comb[i] = list(itertools.combinations(l,i))
    t = np.ones(var_n)
    u = 1
    for h in range(1,k):
        for i in range(int(math.comb(k,h))):
            for j in range(h):
                t[u] *= (a[int(comb[h][i][j])]-1)
            u += 1
    
    A_ub = np.zeros((ub_n,var_n))
    r = 0
    for i in range(k):
        A_ub[r][0] = -1; A_ub[r][i+1] = 1
        r += 1
    g = 1
    for h in range(1,k-1):
        for i in range(int(math.comb(k,h))):
            for j in range(int(math.comb(k,h+1))):
                f = np.zeros(h)
                for z in range(h):
                    for v in range(h+1):
                        if comb[h+1][j][v] == comb[h][i][z]:
                            f[z] = 1
                            break
                if np.sum(f) == h:
                    A_ub[r][g] = -1; A_ub[r][int(p[h])+j] = 1
                    r += 1
            g += 1
    for i in range(k):
        A_ub[r+i][int(p[k-2])+i] = -1
    
    #print(A_ub)
    
    b_ub = np.zeros(ub_n)
    for i in range(k):
        b_ub[ub_n-1-i] = -1
        
    #print(b_ub)
    
    A_eq = np.zeros((k,var_n))
    for s in range(k):
        A_eq[s][0] = 1
        c = 1
        for h in range(1,k):
            for i in range(int(math.comb(k,h))):
                ff = 0
                for j in range(h):
                    if comb[h][i][j] == s:
                        ff = 1
                        break
                if ff == 1:
                    A_eq[s][c] = -(t[c]*math.exp(eps[s]))/(a[s]-1)
                else:
                    A_eq[s][c] = t[c]
                c += 1
    #print(A_eq)
    
    b_eq = np.zeros(k)
    st = 1
    for i in range(k):
        st *= (a[i]-1)
    for s in range(k):
        b_eq[s] = (st*math.exp(eps[s]))/(a[s]-1)
    
    #print(b_eq)
    
    d = np.zeros(var_n); d[0] = 1
    
    ans = optimize.linprog(d,A_ub,b_ub,A_eq,b_eq)
    
    print(ans.status)
    print(ans.success)
    print(ans.message)
        
    return ans.x

In [3]:
def k_LP_reduction(k,a,eps):
    var_n = k+1; ub_n = 2*k
    
    A_ub = np.zeros((ub_n,var_n))
    r = 0
    for i in range(k):
        A_ub[r][0] = -1; A_ub[r][i+1] = 1
        r += 1
    for i in range(k):
        A_ub[r][i+1] = -1
        r += 1
    
    #print(A_ub)
    
    b_ub = np.zeros(ub_n)
    for i in range(k):
        b_ub[ub_n-1-i] = -1
    
    #print(b_ub)
    
    A_eq = np.zeros((k,var_n))
    for s in range(k):
        A_eq[s][0] = 1
        for h in range(1,var_n):
            if s+1 == h:
                A_eq[s][h] = -math.exp(eps[s])
            else:
                A_eq[s][h] = a[s]-1
    #print(A_eq)
    
    b_eq = np.zeros(k)
    st = 1
    for i in range(k):
        st += a[i]
    for s in range(k):
        b_eq[s] = (st/a[s] - 1)*math.exp(eps[s]) - st/a[s] + np.sum(a) - a[s] - k + 2
        
    #print(b_eq)
    
    d = np.zeros(var_n); d[0] = 1
    
    ans = optimize.linprog(d,A_ub,b_ub,A_eq,b_eq)
    
    print(ans.status)
    print(ans.success)
    print(ans.message)
        
    return ans.x

In [4]:
print("Original ver.")
print(k_LP_original(10,[2,3,2,4,6,5,3,2,7,4],[math.log(13),math.log(12),math.log(14),math.log(12),math.log(13),math.log(13),math.log(14),math.log(13),math.log(14),math.log(12)]))
print("---------------")
print("Reduction ver.")
print(k_LP_reduction(10,[2,3,2,4,6,5,3,2,7,4],[math.log(13),math.log(12),math.log(14),math.log(12),math.log(13),math.log(13),math.log(14),math.log(13),math.log(14),math.log(12)]))

Original ver.
0
True
Optimization terminated successfully.
[6.53133823e+04 4.58437226e+04 6.53133823e+04 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
---------------
Reduction ver.
2
False
The algorithm terminated successfully and determined that the problem is infeasible.
[5.53044142 1.76670706 1.98559529 1.74181486 2.079874   2.15098464
 2.11260386 1.93560648 1.76670706 2.15974012 2.079874  ]
